In [ ]:
import numpy as np
from PIL import Image
import time
import pandas as pd
import cupy as cp
from scipy.signal import medfilt2d

In [ ]:
img = Image.open('Noise_salt_and_pepper.bmp')
arr = cp.array(img)
print(arr.shape)

In [117]:
def init(matrix_size: tuple, matrix: cp.ndarray, pars: dict):
    add_kernel = cp.RawKernel(r'''
            extern "C"
            __global__ void saltandpepper(unsigned char* input, unsigned char* output, int width, int height) {
                int x =  threadIdx.x + blockIdx.x * blockDim.x;
                int y = threadIdx.y + blockIdx.y * blockDim.y;

                if (x < width && y < height) {
                    unsigned char window[9];
                    int k = 0;

                    for (int i = -1; i <= 1; i++) {
                        for (int j = -1; j <= 1; j++) {
                            int nx = x + i;
                            int ny = y + j;

                            if (nx >= 0 && nx < width && ny >= 0 && ny < height) {
                                window[k] = input[nx + ny * width];
                            } else {
                                window[k] = 0;
                            }

                            k++;
                        }
                    }

                    for (int i = 0; i < 9; i++) {
                        for (int j = i + 1; j < 9; j++) {
                            if (window[i] > window[j]) {
                                unsigned char temp = window[i];
                                window[i] = window[j];
                                window[j] = temp;
                            }
                        }
                    }

                    output[x + y * width] = window[4];
                }
            }
        ''',
        "saltandpepper")
    shape = matrix_size
    matrix_scipy = cp.asarray(matrix)
    matrix_func = cp.asarray(matrix)
    matrix = cp.asarray(matrix.flatten())
    params = pars
    result = cp.zeros((shape[0] * shape[1]), dtype=cp.uint8)
    return add_kernel, shape, matrix, matrix_scipy, matrix_func, params, result


def salt_and_pepper(matrix_func, shape):
  out_im = np.zeros(shape, dtype=np.uint8)
  window = []
  for i in range(shape[0]):
    for j in range(shape[1]):
      if (i <= 0 or j <= 0 or i >= shape[0] - 1 or j >= shape[1] - 1):
        continue
      window = [matrix_func[i+x, j+y] for x in range(-1, 2) for y in range(-1, 2)]
      window.sort()
      out_im[i][j] = window[4]
      window = []
  return out_im


def getresult(add_kernel, params, result, shape, matrix, matrix_scipy, matrix_func):
    gpustart = time.perf_counter()
    result_gpu = add_kernel((params["blockX"], params["blockY"]),(params["gridX"], params["gridY"]),(matrix, result,shape[1], shape[0]))
    gpuend = time.perf_counter()

    scipystart = time.perf_counter()
    result_scipy = medfilt2d(matrix_scipy.get(), kernel_size=3)
    scipyend = time.perf_counter()

    funcstart = time.perf_counter()
    result_cpu_func = salt_and_pepper(matrix_func, shape)
    funcend = time.perf_counter()

    img_gpu = Image.fromarray(result.get().reshape(shape))
    img_scipy = Image.fromarray(result_scipy)
    img_func = Image.fromarray(result_cpu_func)

    img_gpu.save('gpuim.bmp')
    img_scipy.save('scipyim.bmp')
    img_func.save('funcim.bmp')

    print(f'параметры: {str(params)}')

    return  (gpuend - gpustart), (scipyend - scipystart), (funcend - funcstart)

In [ ]:
size = arr.shape
block = (32, 32)
grid = (round((size[1] + block[0] - 1) / block[0]), round((size[0] + block[1] - 1) / block[1]))
add_kernel, shape, matrix, matrix_scipy, matrix_func, params, result = init(size, arr, {"blockX": block[0],"blockY": block[1],"gridX": grid[0],"gridY": grid[1],})
timegpu,timescipy,timefunc = getresult(add_kernel, params, result, shape, matrix, matrix_scipy, matrix_func)

In [ ]:
print(f'gpu time: {timegpu} c')
print(f'func time: {timefunc} c')
print(f'scipy time: {timescipy} c')
print(f'gpu vs func: {timefunc/timegpu} раз')
print(f'gpu vs scipy: {timescipy/timegpu} раз')
print(f'scipy vs func: {timefunc/timescipy} раз')